In [1]:
!pip install scikit-learn tensorflow

   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 2.4/332.0 MB 11.9 MB/s eta 0:00:28
    --------------------------------------- 5.0/332.0 MB 11.8 MB/s eta 0:00:28
    --------------------------------------- 7.3/332.0 MB 11.8 MB/s eta 0:00:28
   - -------------------------------------- 9.4/332.0 MB 11.4 MB/s eta 0:00:29
   - -------------------------------------- 11.8/332.0 MB 11.4 MB/s eta 0:00:28
   - -------------------------------------- 14.4/332.0 MB 11.5 MB/s eta 0:00:28
   -- ------------------------------------- 16.8/332.0 MB 11.5 MB/s eta 0:00:28
   -- ------------------------------------- 19.1/332.0 MB 11.6 MB/s eta 0:00:28
   -- ------------------------------------- 21.8/332.0 MB 11.6 MB/s eta 0:00:27
   -- ------------------------------------- 24.1/332.0 MB 11.6 MB/s eta 0:00:27
   --- ------------------------------------ 26.5/332.0 MB 11.6 MB/s eta 0:00:27
   --- ------------------------------------ 29.1/332.

In [3]:
import scipy.io as io
import numpy as np

mat_path = r"C:\Users\이정민\Desktop\breast cancer\breast_maxbbox_3d_fastpeak.mat"

d = io.loadmat(mat_path)
vols = d["vols"]        # (N, X, Y, Z)
labels = d["labels"]    # mat에 따라 (N,1) or (1,N) or (N,)

# labels shape 정리
labels = np.array(labels).squeeze()

print("vols:", vols.shape, vols.dtype)
print("labels:", labels.shape, labels.dtype)
print("label unique:", np.unique(labels))

vols: (200, 182, 212, 120) uint16
labels: (200,) int64
label unique: [0 1 2 3]


In [4]:
vols = vols.astype(np.float32)

# (N, X, Y, Z) -> (N, X, Y, Z, 1)
if vols.ndim == 4:
    vols = vols[..., np.newaxis]

print("vols after channel:", vols.shape)  # (N, X, Y, Z, 1)

vols after channel: (200, 182, 212, 120, 1)


In [5]:
from sklearn.model_selection import train_test_split

X = vols
y = labels

# 70/15/15 split
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=123, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=123, stratify=y_temp
)

print("Train:", X_train.shape, y_train.shape)
print("Val  :", X_val.shape, y_val.shape)
print("Test :", X_test.shape, y_test.shape)

Train: (140, 182, 212, 120, 1) (140,)
Val  : (30, 182, 212, 120, 1) (30,)
Test : (30, 182, 212, 120, 1) (30,)


In [6]:
train_mean = X_train.mean()
train_std = X_train.std() + 1e-6

X_train = (X_train - train_mean) / train_std
X_val   = (X_val   - train_mean) / train_std
X_test  = (X_test  - train_mean) / train_std

print("standardized. mean/std (train):", X_train.mean(), X_train.std())

standardized. mean/std (train): 7.754416e-06 1.0000358


In [7]:
import tensorflow as tf

num_classes = 4  # 0~3
y_train_oh = tf.keras.utils.to_categorical(y_train, num_classes)
y_val_oh   = tf.keras.utils.to_categorical(y_val, num_classes)
y_test_oh  = tf.keras.utils.to_categorical(y_test, num_classes)

C:\Anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [8]:
from tensorflow.keras import layers, models

input_shape = X_train.shape[1:]  # (X, Y, Z, 1)

def build_small_3dcnn(input_shape, num_classes=4):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv3D(16, 3, padding="same", activation="relu")(inputs)
    x = layers.MaxPool3D(2)(x)

    x = layers.Conv3D(32, 3, padding="same", activation="relu")(x)
    x = layers.MaxPool3D(2)(x)

    x = layers.Conv3D(64, 3, padding="same", activation="relu")(x)
    x = layers.GlobalAveragePooling3D()(x)

    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs, outputs)
    return model

model = build_small_3dcnn(input_shape, num_classes)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 182, 212, 120, 1)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d (Conv3D)                      │ (None, 182, 212, 120, 16)   │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d (MaxPooling3D)         │ (None, 91, 106, 60, 16)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 91, 106, 60, 32)     │          13,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_1 (MaxPooling3D)       │ (None, 45, 53, 30, 32)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_2 (Conv3D)                    │ (None, 45, 53, 30, 64)      │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling3d             │ (None, 64)                  │               0 │
│ (GlobalAveragePooling3D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 69,924 (273.14 KB)

 Trainable params: 69,924 (273.14 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y_train)
cw = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weight = {int(c): float(w) for c, w in zip(classes, cw)}
print("class_weight:", class_weight)

class_weight: {0: 0.5833333333333334, 1: 1.0294117647058822, 2: 1.4, 3: 1.6666666666666667}


In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3)
]

In [11]:
history = model.fit(
    X_train, y_train_oh,
    validation_data=(X_val, y_val_oh),
    epochs=30,
    batch_size=2,      # CPU면 2~8 사이 권장
    class_weight=class_weight,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.1857 - loss: 1.4113 - val_accuracy: 0.2333 - val_loss: 1.3867 - learning_rate: 0.0010
Epoch 2/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.1571 - loss: 1.3869 - val_accuracy: 0.2000 - val_loss: 1.3871 - learning_rate: 0.0010
Epoch 3/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.1643 - loss: 1.3869 - val_accuracy: 0.2000 - val_loss: 1.3873 - learning_rate: 0.0010
Epoch 4/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - accuracy: 0.1571 - loss: 1.3868 - val_accuracy: 0.2000 - val_loss: 1.3873 - learning_rate: 0.0010
Epoch 5/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.1786 - loss: 1.3866 - val_accuracy: 0.2000 - val_loss: 1.3877 - learning_rate: 5.0000e-04
Epoch 6/30
70/70 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.2071 - loss: 1.3873 - val_accuracy: 0.2000 - val_loss: 1.3872 - learning_rate: 5.0000e-04


In [12]:
from sklearn.metrics import classification_report, confusion_matrix

# 예측
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

print("Classification Report (test):")
print(classification_report(y_test, y_pred, digits=4))

print("Confusion Matrix (test):")
print(confusion_matrix(y_test, y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 214s 214s/step
Classification Report (test):
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        13
           1     0.2667    1.0000    0.4211         8
           2     0.0000    0.0000    0.0000         5
           3     0.0000    0.0000    0.0000         4

    accuracy                         0.2667        30
   macro avg     0.0667    0.2500    0.1053        30
weighted avg     0.0711    0.2667    0.1123        30

Confusion Matrix (test):
[[ 0 13  0  0]
 [ 0  8  0  0]
 [ 0  5  0  0]
 [ 0  4  0  0]]


C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
